In [1]:
import keras
from keras.utils import to_categorical
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.layers import Dense, Flatten, Embedding
import numpy as np
import re
from bpe import BPE

In [2]:
# Hyperparameters
vocab_size = 2000
max_length = 20

In [3]:
# text cleaning and fit bpe tokenizer
bpe = BPE(vocab_size)

def tokenize(text):
    clean_text = re.sub("r[^A-z0-9?!.', ]", '', str(text))
    return ' '.join(word_tokenize(clean_text))


x = []
y = []

with open("dialogs.txt", 'r') as file:
    for line in file.readlines():
        question, answer = line.replace("\n", '').split('\t')
        x.append(tokenize(question))
        y.append(answer)


bpe.fit(x)

x = [bpe.decode(i) for i in x]
x = pad_sequences(x, maxlen=max_length)

In [4]:
# Save the tokenizer as a file
bpe.save()

# Load the tokenizer from a file
bpe.load()

In [5]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
output_size = len(label_encoder.classes_)
y = to_categorical(y)

In [6]:
test_x, test_y = x[:100], y[:100]

In [7]:
# Model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=32, input_length=max_length),
    Flatten(),
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=output_size, activation='softmax')
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 32)            64000     
                                                                 
 flatten (Flatten)           (None, 640)               0         
                                                                 
 dense (Dense)               (None, 64)                41024     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 3608)              61336     
                                                                 
Total params: 168968 (660.03 KB)
Trainable params: 16896

In [8]:
model.fit(x, y, epochs=50)

val_loss, val_acc = model.evaluate(test_x, test_y)
print(val_loss)
print(val_acc)

Epoch 1/50
120/120 [==============================] - 1s 3ms/step - loss: 8.2098 - accuracy: 0.0026
Epoch 2/50
120/120 [==============================] - 0s 3ms/step - loss: 8.1921 - accuracy: 0.0052
Epoch 3/50
120/120 [==============================] - 0s 3ms/step - loss: 8.1896 - accuracy: 0.0047
Epoch 4/50
120/120 [==============================] - 0s 2ms/step - loss: 8.0789 - accuracy: 0.0060
Epoch 5/50
120/120 [==============================] - 0s 3ms/step - loss: 7.5165 - accuracy: 0.0058
Epoch 6/50
120/120 [==============================] - 0s 3ms/step - loss: 6.9175 - accuracy: 0.0076
Epoch 7/50
120/120 [==============================] - 0s 2ms/step - loss: 6.3319 - accuracy: 0.0136
Epoch 8/50
120/120 [==============================] - 0s 3ms/step - loss: 5.7210 - accuracy: 0.0227
Epoch 9/50
120/120 [==============================] - 0s 2ms/step - loss: 5.0635 - accuracy: 0.0442
Epoch 10/50
120/120 [==============================] - 0s 2ms/step - loss: 4.3962 - accuracy: 0.0816

In [9]:
# save model 
model.save("ChatBot.hdf5")

model = keras.models.load_model("ChatBot.hdf5")
val_loss, val_acc = model.evaluate(test_x, test_y)
print(val_loss)
print(val_acc)

e:\Projects\Python\ChatBot\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 2ms/step - loss: 0.0964 - accuracy: 0.9700
0.09635287523269653
0.9700000286102295


In [10]:
def preprocess_input(text):
    text_lower = text.lower()
    tokenize_text = tokenize(text_lower)
    decode_text = bpe.decode(tokenize_text)
    return pad_sequences([decode_text], maxlen=max_length)


while True:
    user_input = input("You: ")
    print(f"You: {user_input}")
    preprocessed_input = preprocess_input(user_input)
    prediction = model.predict(preprocessed_input)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    print(f"Bot: {predicted_label[0]}")
    print()

1/1 [==============================] - 0s 73ms/step
Bot: i'm fine. how about yourself?

1/1 [==============================] - 0s 15ms/step
Bot: i haven't been better. how about yourself?

1/1 [==============================] - 0s 15ms/step
Bot: i'm going to pcc.

1/1 [==============================] - 0s 20ms/step
Bot:  bye

1/1 [==============================] - 0s 16ms/step
Bot:  bye

1/1 [==============================] - 0s 16ms/step
Bot:  bye

